In [1]:
import pickle
import numpy as np

from prompt_creation import PromptLlamaMPC

In [2]:
prompt_configuration = 'SCHEME0_OUTPUT0.pkl'

dataset_sample = '/home/maryam/PrespectivePaper/Data/wikipedia.pkl'

In [3]:
with open(prompt_configuration, 'rb') as f:
    prompt = pickle.load(f)
    
with open(dataset_sample, 'rb') as f:
    data = pickle.load(f)

In [4]:
prompt_generator = PromptLlamaMPC(prompt)

In [5]:
def conversation_basic(posts, speakers):
    history = ""
    
    i = 1

    for p, s in zip(posts, speakers):
        history += str(i) + " : " + s + " [WRITES] " + p + "\n"
        i = i + 1

    history = history[:-1]

    return history

In [6]:
def conversation_profile(speakers, addrs, user_tag):
    history = ""

    last_diff_user = 16
    last_user = 16
    
    i = 1

    for s, a in zip(speakers, addrs):
        final_a = a

        if last_user != s:
            last_diff_user = last_user
        if a == 17:
            final_a = last_diff_user
        last_user = s

        if final_a == 16:
            final_a = -1

        history += str(i) + " : " + user_tag[s] + " to " + user_tag[final_a] + "\n"
        
        i = i + 1

    history = history[:-1]

    return history

In [7]:
basic_conv = list()
profile_conv = list()

for s in data:
    print(s)
    basic_conv.append(conversation_basic(s['context'], s['speaker']))
    profile_conv.append(conversation_profile(s['context'], s['speaker']))

524288


TypeError: string indices must be integers, not 'str'

# RESPONSE SELECTION PROMPT

In [8]:
rs_conv_prompts = []
rs_conv_profile_prompts = []

user_tag = ["[ALEX]", "[BENNY]", "[CAM]", "[DANA]", "[OTHER]"]

for b_c, s_c, summ, desc in zip(basic_conv, profile_conv):
    
    item = dict()
    item['conversation'] = b_c
    item['interaction'] = s_c
    item['summary'] = summ
    item['description'] = desc


    rs_conv_prompts.append(prompt_generator.response_selection(conversation=True, item=item)[0])
    rs_conv_profile_prompts.append(prompt_generator.response_selection(conversation=True, profile=True, item=item)[0])


with open('prompt/rs_conv_prompts.pkl', 'wb') as f:
    pickle.dump(rs_conv_prompts, f)
    
with open('prompt/rs_conv_profile_prompts.pkl', 'wb') as f:
    pickle.dump(rs_conv_profile_prompts, f)

FileNotFoundError: [Errno 2] No such file or directory: 'prompt/rs_conv_prompts.pkl'